In [68]:
import pandas as pd

In [69]:
import urllib
from bs4 import BeautifulSoup
import re

**Завдання 2. (1 бал)**

In [136]:
url = r'http://d396qusza40orc.cloudfront.net/getdata%2Fdata%2Frestaurants.xml'
data = urllib.request.urlopen(url).read()
soup = BeautifulSoup(data)

Скільки ресторанів мають zipcode 21231? 

In [141]:
count = 0
for restraunt in soup.select('row'):
    if restraunt.find('zipcode').text == '21231':
        count += 1
count

127

**Завдання 3. (8 балів)**

In [93]:
racers = pd.DataFrame(columns=[
    'Position', 
    'Points', 
    'Name', 
    'Surname', 
    'Country', 
    'Team', 
    'Photo', 
    'Number', 
    'Profile URL', 
    'Biography', 
    'Podiums', 
    'Points', 
    'Grands Prix entered', 
    'World Championships', 
    'Highest race finish',
    'Highest grid position', 
    'Date of birth', 
    'Place of birth',
])

Виконуємо HTTP-запит, завантажуємо HTML документ та перетвоюємо його на сутність BeautifulSoup бібліотеки bs4 для подальшого парсингу.

In [75]:
base_url = r'https://www.formula1.com'
url = r'https://www.formula1.com/en/drivers.html'
data = urllib.request.urlopen(url).read()
soup = BeautifulSoup(data, 'html.parser')
racers_html = soup.select('div.col-12.col-md-6.col-lg-4')

Парсимо HTML за допомогою бібліотеки. Значення, що знаходяться на окремій сторінці, отримуємо окремим посиланням, яке знаходимо у відповідному елементі.

In [103]:
for i in range(len(racers_html)):
    div = racers_html[i]
    racers.at[i, 'Position'] = div.find('div', class_='rank').text
#     racers.at[i, 'Points'] = div.find('div', class_='f1-wide--s').text  # is duplicated
    racers.at[i, 'Name'] = div.find('span', class_='f1--xxs').text
    racers.at[i, 'Surname'] = div.find('span', class_='f1-bold--s').text
    
    country_link = div.find('picture', class_='coutnry-flag--photo')
    if country_link:
        country_link = country_link.find('img').get('data-src')
        country_link = re.findall(r'https://www.formula1.com/content/dam/fom-website/flags/(.*?).jpg.transform/2col/image.jpg', country_link)[0]
        country_link = country_link.replace('%20', ' ')
    racers.at[i, 'Country'] = country_link
    
    racers.at[i, 'Team'] = div.find('p', class_='listing-item--team').text
    photo_picture = div.find('picture', class_='listing-item--photo')
    racers.at[i, 'Photo'] = photo_picture.find('img').get('data-src')
    number_picture = div.find('picture', class_='listing-item--number')
    racers.at[i, 'Number'] = number_picture.find('img').get('data-src')
        
    racer_data = base_url + div.find('a', class_='listing-item--link').get('href')
    racers.at[i, 'Profile URL'] = racer_data
    racer_data = urllib.request.urlopen(racer_data).read()
    racer_data = BeautifulSoup(racer_data, 'html.parser')
    
    racer_stats = racer_data.find('table', class_='stat-list')
    for stat in racer_stats.select('tr'):
        stat_name = stat.find('span').text
        stat_value = stat.find('td').text
        if stat_name in racers.columns:
            racers.at[i, stat_name] = stat_value
    
    racers.at[i, 'Biography'] = racer_data.find('section', class_='biography').find('p').text
    
racers

,Position,Points,Name,Surname,Country,Team,Photo,Number,Profile URL,Biography,Podiums,Points,Grands Prix entered,World Championships,Highest race finish,Highest grid position,Date of birth,Place of birth
0,1,3872,Lewis,Hamilton,United Kingdom,Mercedes,https://www.formula1.com/content/dam/fom-websi...,https://www.formula1.com/content/dam/fom-websi...,https://www.formula1.com/en/drivers/lewis-hami...,‘Still I Rise’ – these are the words emblazone...,169,3872,270,7,1 (x98),1,07/01/1985,"Stevenage, England"
1,2,1242,Max,Verstappen,Netherlands,Red Bull Racing,https://www.formula1.com/content/dam/fom-websi...,https://www.formula1.com/content/dam/fom-websi...,https://www.formula1.com/en/drivers/max-versta...,"He’s Max by name, and max by nature.",46,1242,123,N/A,1 (x11),1,30/09/1997,"Hasselt, Belgium"
2,3,1559,Valtteri,Bottas,Finland,Mercedes,https://www.formula1.com/content/dam/fom-websi...,https://www.formula1.com/content/dam/fom-websi...,https://www.formula1.com/en/drivers/valtteri-b...,Learning his craft on Finnish roads of ice and...,59,1559,161,N/A,1 (x9),1,28/08/1989,"Nastola, Finland"
3,4,187,Lando,Norris,United Kingdom,McLaren,https://www.formula1.com/content/dam/fom-websi...,https://www.formula1.com/content/dam/fom-websi...,https://www.formula1.com/en/drivers/lando-norr...,Lando Norris may not be named after Star Wars ...,2,187,42,N/A,3 (x2),3,13/11/1999,"Bristol, England"
4,5,441,Charles,Leclerc,Monaco,Ferrari,https://www.formula1.com/content/dam/fom-websi...,https://www.formula1.com/content/dam/fom-websi...,https://www.formula1.com/en/drivers/charles-le...,"Born in the Mediterranean idyll of Monaco, Lec...",12,441,63,N/A,1 (x2),1,16/10/1997,"Monte Carlo, Monaco"
5,6,738,Sergio,Perez,Mexico,Red Bull Racing,https://www.formula1.com/content/dam/fom-websi...,https://www.formula1.com/content/dam/fom-websi...,https://www.formula1.com/en/drivers/sergio-per...,He’s the fighter with a gentle touch from the ...,10,738,197,N/A,1 (x1),2,26/01/1990,"Guadalajara, Mexico"
6,7,1183,Daniel,Ricciardo,Australia,McLaren,https://www.formula1.com/content/dam/fom-websi...,https://www.formula1.com/content/dam/fom-websi...,https://www.formula1.com/en/drivers/daniel-ric...,The self-styled “Honey Badger” is fuzzy on the...,31,1183,192,N/A,1 (x7),1,01/07/1989,"Perth, Australia"
7,8,392,Carlos,Sainz,Spain,Ferrari,https://www.formula1.com/content/dam/fom-websi...,https://www.formula1.com/content/dam/fom-websi...,https://www.formula1.com/en/drivers/carlos-sai...,He’s the matador from Madrid racing royalty.,2,392,123,N/A,2 (x1),3,01/09/1994,"Madrid, Spain"
8,9,208,Esteban,Ocon,France,Alpine,https://www.formula1.com/content/dam/fom-websi...,https://www.formula1.com/content/dam/fom-websi...,https://www.formula1.com/en/drivers/esteban-oc...,If there’s one word that dominates Esteban Oco...,1,208,71,N/A,2 (x1),3,17/09/1996,"Évreux, Normandy"
9,10,207,Pierre,Gasly,France,AlphaTauri,https://www.formula1.com/content/dam/fom-websi...,https://www.formula1.com/content/dam/fom-websi...,https://www.formula1.com/en/drivers/pierre-gas...,If there’s one man who knows how big a rollerc...,2,207,68,N/A,1 (x1),4,07/02/1996,"Rouen, France"


1. Виведіть перших 10 гонщиків датафрейму.

In [104]:
racers[:10]

,Position,Points,Name,Surname,Country,Team,Photo,Number,Profile URL,Biography,Podiums,Points,Grands Prix entered,World Championships,Highest race finish,Highest grid position,Date of birth,Place of birth
0,1,3872,Lewis,Hamilton,United Kingdom,Mercedes,https://www.formula1.com/content/dam/fom-websi...,https://www.formula1.com/content/dam/fom-websi...,https://www.formula1.com/en/drivers/lewis-hami...,‘Still I Rise’ – these are the words emblazone...,169,3872,270,7,1 (x98),1,07/01/1985,"Stevenage, England"
1,2,1242,Max,Verstappen,Netherlands,Red Bull Racing,https://www.formula1.com/content/dam/fom-websi...,https://www.formula1.com/content/dam/fom-websi...,https://www.formula1.com/en/drivers/max-versta...,"He’s Max by name, and max by nature.",46,1242,123,N/A,1 (x11),1,30/09/1997,"Hasselt, Belgium"
2,3,1559,Valtteri,Bottas,Finland,Mercedes,https://www.formula1.com/content/dam/fom-websi...,https://www.formula1.com/content/dam/fom-websi...,https://www.formula1.com/en/drivers/valtteri-b...,Learning his craft on Finnish roads of ice and...,59,1559,161,N/A,1 (x9),1,28/08/1989,"Nastola, Finland"
3,4,187,Lando,Norris,United Kingdom,McLaren,https://www.formula1.com/content/dam/fom-websi...,https://www.formula1.com/content/dam/fom-websi...,https://www.formula1.com/en/drivers/lando-norr...,Lando Norris may not be named after Star Wars ...,2,187,42,N/A,3 (x2),3,13/11/1999,"Bristol, England"
4,5,441,Charles,Leclerc,Monaco,Ferrari,https://www.formula1.com/content/dam/fom-websi...,https://www.formula1.com/content/dam/fom-websi...,https://www.formula1.com/en/drivers/charles-le...,"Born in the Mediterranean idyll of Monaco, Lec...",12,441,63,N/A,1 (x2),1,16/10/1997,"Monte Carlo, Monaco"
5,6,738,Sergio,Perez,Mexico,Red Bull Racing,https://www.formula1.com/content/dam/fom-websi...,https://www.formula1.com/content/dam/fom-websi...,https://www.formula1.com/en/drivers/sergio-per...,He’s the fighter with a gentle touch from the ...,10,738,197,N/A,1 (x1),2,26/01/1990,"Guadalajara, Mexico"
6,7,1183,Daniel,Ricciardo,Australia,McLaren,https://www.formula1.com/content/dam/fom-websi...,https://www.formula1.com/content/dam/fom-websi...,https://www.formula1.com/en/drivers/daniel-ric...,The self-styled “Honey Badger” is fuzzy on the...,31,1183,192,N/A,1 (x7),1,01/07/1989,"Perth, Australia"
7,8,392,Carlos,Sainz,Spain,Ferrari,https://www.formula1.com/content/dam/fom-websi...,https://www.formula1.com/content/dam/fom-websi...,https://www.formula1.com/en/drivers/carlos-sai...,He’s the matador from Madrid racing royalty.,2,392,123,N/A,2 (x1),3,01/09/1994,"Madrid, Spain"
8,9,208,Esteban,Ocon,France,Alpine,https://www.formula1.com/content/dam/fom-websi...,https://www.formula1.com/content/dam/fom-websi...,https://www.formula1.com/en/drivers/esteban-oc...,If there’s one word that dominates Esteban Oco...,1,208,71,N/A,2 (x1),3,17/09/1996,"Évreux, Normandy"
9,10,207,Pierre,Gasly,France,AlphaTauri,https://www.formula1.com/content/dam/fom-websi...,https://www.formula1.com/content/dam/fom-websi...,https://www.formula1.com/en/drivers/pierre-gas...,If there’s one man who knows how big a rollerc...,2,207,68,N/A,1 (x1),4,07/02/1996,"Rouen, France"


2. Виведіть всіх гонщиків із США.

In [108]:
racers[racers['Country'] == 'United States']

,Position,Points,Name,Surname,Country,Team,Photo,Number,Profile URL,Biography,Podiums,Points,Grands Prix entered,World Championships,Highest race finish,Highest grid position,Date of birth,Place of birth


Перевірка:

In [109]:
racers[racers['Country'] == 'France']

,Position,Points,Name,Surname,Country,Team,Photo,Number,Profile URL,Biography,Podiums,Points,Grands Prix entered,World Championships,Highest race finish,Highest grid position,Date of birth,Place of birth
8,9,208,Esteban,Ocon,France,Alpine,https://www.formula1.com/content/dam/fom-websi...,https://www.formula1.com/content/dam/fom-websi...,https://www.formula1.com/en/drivers/esteban-oc...,If there’s one word that dominates Esteban Oco...,1,208,71,N/A,2 (x1),3,17/09/1996,"Évreux, Normandy"
9,10,207,Pierre,Gasly,France,AlphaTauri,https://www.formula1.com/content/dam/fom-websi...,https://www.formula1.com/content/dam/fom-websi...,https://www.formula1.com/en/drivers/pierre-gas...,If there’s one man who knows how big a rollerc...,2,207,68,N/A,1 (x1),4,07/02/1996,"Rouen, France"


3. Скільки гонщиків брали участь у «Чемпіонаті світу»

In [131]:
wc = racers[['World Championships']].replace('N/A', 0)
wc['World Championships'] = pd.to_numeric(wc['World Championships'])
len(wc[wc['World Championships'] > 0])

4

In [132]:
racers

,Position,Points,Name,Surname,Country,Team,Photo,Number,Profile URL,Biography,Podiums,Points,Grands Prix entered,World Championships,Highest race finish,Highest grid position,Date of birth,Place of birth
0,1,3872,Lewis,Hamilton,United Kingdom,Mercedes,https://www.formula1.com/content/dam/fom-websi...,https://www.formula1.com/content/dam/fom-websi...,https://www.formula1.com/en/drivers/lewis-hami...,‘Still I Rise’ – these are the words emblazone...,169,3872,270,7,1 (x98),1,07/01/1985,"Stevenage, England"
1,2,1242,Max,Verstappen,Netherlands,Red Bull Racing,https://www.formula1.com/content/dam/fom-websi...,https://www.formula1.com/content/dam/fom-websi...,https://www.formula1.com/en/drivers/max-versta...,"He’s Max by name, and max by nature.",46,1242,123,N/A,1 (x11),1,30/09/1997,"Hasselt, Belgium"
2,3,1559,Valtteri,Bottas,Finland,Mercedes,https://www.formula1.com/content/dam/fom-websi...,https://www.formula1.com/content/dam/fom-websi...,https://www.formula1.com/en/drivers/valtteri-b...,Learning his craft on Finnish roads of ice and...,59,1559,161,N/A,1 (x9),1,28/08/1989,"Nastola, Finland"
3,4,187,Lando,Norris,United Kingdom,McLaren,https://www.formula1.com/content/dam/fom-websi...,https://www.formula1.com/content/dam/fom-websi...,https://www.formula1.com/en/drivers/lando-norr...,Lando Norris may not be named after Star Wars ...,2,187,42,N/A,3 (x2),3,13/11/1999,"Bristol, England"
4,5,441,Charles,Leclerc,Monaco,Ferrari,https://www.formula1.com/content/dam/fom-websi...,https://www.formula1.com/content/dam/fom-websi...,https://www.formula1.com/en/drivers/charles-le...,"Born in the Mediterranean idyll of Monaco, Lec...",12,441,63,N/A,1 (x2),1,16/10/1997,"Monte Carlo, Monaco"
5,6,738,Sergio,Perez,Mexico,Red Bull Racing,https://www.formula1.com/content/dam/fom-websi...,https://www.formula1.com/content/dam/fom-websi...,https://www.formula1.com/en/drivers/sergio-per...,He’s the fighter with a gentle touch from the ...,10,738,197,N/A,1 (x1),2,26/01/1990,"Guadalajara, Mexico"
6,7,1183,Daniel,Ricciardo,Australia,McLaren,https://www.formula1.com/content/dam/fom-websi...,https://www.formula1.com/content/dam/fom-websi...,https://www.formula1.com/en/drivers/daniel-ric...,The self-styled “Honey Badger” is fuzzy on the...,31,1183,192,N/A,1 (x7),1,01/07/1989,"Perth, Australia"
7,8,392,Carlos,Sainz,Spain,Ferrari,https://www.formula1.com/content/dam/fom-websi...,https://www.formula1.com/content/dam/fom-websi...,https://www.formula1.com/en/drivers/carlos-sai...,He’s the matador from Madrid racing royalty.,2,392,123,N/A,2 (x1),3,01/09/1994,"Madrid, Spain"
8,9,208,Esteban,Ocon,France,Alpine,https://www.formula1.com/content/dam/fom-websi...,https://www.formula1.com/content/dam/fom-websi...,https://www.formula1.com/en/drivers/esteban-oc...,If there’s one word that dominates Esteban Oco...,1,208,71,N/A,2 (x1),3,17/09/1996,"Évreux, Normandy"
9,10,207,Pierre,Gasly,France,AlphaTauri,https://www.formula1.com/content/dam/fom-websi...,https://www.formula1.com/content/dam/fom-websi...,https://www.formula1.com/en/drivers/pierre-gas...,If there’s one man who knows how big a rollerc...,2,207,68,N/A,1 (x1),4,07/02/1996,"Rouen, France"
